In [4]:
# !pip install easydict

  Using cached easydict-1.9-py3-none-any.whl


In [1]:
import gc
import cv2
import torchvision
import torch
import pandas as pd
import numpy as np

import os
import os.path as osp

import argparse
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np
from easydict import EasyDict

/opt/conda/envs/openmmlab/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
# exp_dir = 'sparse_rcnn_r50_fpn_300_proposals_crop_mstrain_704-1024_36_coco'
# exp_name = 'sparse_rcnn_r50_fpn_300_proposals_crop_mstrain_704-1024_36_coco.py'
# cfg_path = f'../mmdetection/configs/{exp_dir}/{exp_name}'
# print(cfg_path)


In [2]:
def load_dataloader(exp_dir,exp_name,checkpoint_path):
    cfg_path = f'../mmdetection/configs/{exp_dir}/{exp_name}'
    config = cfg_path
    
    args = EasyDict({
        'config' : config,
        'checkpoint': checkpoint_path,
        'score_thr' :0.001,
        'iou_thr': 0.5, 
        'work_dir': None,
        'seed': 42
    })

    cfg = Config.fromfile(args.config)
    seed = args.seed
    cfg.gpu_ids = [1]

    cfg.work_dir = osp.join('../work_dirs',osp.splitext(osp.basename(args.config))[0])
    print(cfg.work_dir)
    cfg.data.test.test_mode = True
    

    cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
    cfg.model.train_cfg = None
    cfg.model.test_cfg.rcnn.score_thr = args.score_thr
    cfg.model.test_cfg.rcnn.iou_threshold = args.iou_thr
    
    cfg.data.test['ann_file'] = os.path.join('../',cfg.data.test['ann_file'])
    cfg.data.test['img_prefix'] = os.path.join('../',cfg.data.test['img_prefix'])
    print(cfg.data.test)
    
    dataset = build_dataset(cfg.data.test)
    data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
    return data_loader


data_loader = load_dataloader(exp_dir='sparse_rcnn_r50_fpn_300_proposals_crop_mstrain_704-1024_36_coco',
        exp_name='sparse_rcnn_r50_fpn_300_proposals_crop_mstrain_704-1024_36_coco.py',
            checkpoint_path='best_bbox_mAP_50_epoch_22'),

../work_dirs/sparse_rcnn_r50_fpn_300_proposals_crop_mstrain_704-1024_36_coco
{'type': 'CocoDataset', 'ann_file': '../dataset2/test.json', 'img_prefix': '../dataset2/', 'pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (512, 512), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}], 'classes': ('General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing'), 'test_mode': True}
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [10]:
data_loader[0].dataset


CocoDataset Test dataset with number of images 4871, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 0     | 1 [Paper]     | 0     | 2 [Paper pack]  | 0     | 3 [Metal]   | 0     | 4 [Glass]    | 0     |
| 5 [Plastic]       | 0     | 6 [Styrofoam] | 0     | 7 [Plastic bag] | 0     | 8 [Battery] | 0     | 9 [Clothing] | 0     |
|                   |       |               |       |                 |       |             |       |              |       |
| -1 background     | 4871  |               |       |                 |       |             |       |              |       |
+-------------------+-------+---------------+----

In [6]:
for i,data in enumerate(data_loader):
    # print(data.dataset)
    for d in data.dataset:
        print(d)
        print(d[])
        print(d['img'])
        break
    

{'img_metas': [DataContainer({'filename': '../dataset2/test/0000.jpg', 'ori_filename': 'test/0000.jpg', 'ori_shape': (1024, 1024, 3), 'img_shape': (512, 512, 3), 'pad_shape': (512, 512, 3), 'scale_factor': array([0.5, 0.5, 0.5, 0.5], dtype=float32), 'flip': False, 'flip_direction': None, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}})], 'img': [tensor([[[ 0.4337,  0.4337,  0.4337,  ...,  0.5536,  0.1254,  0.7248],
         [ 0.4337,  0.4337,  0.4337,  ...,  0.6906,  0.5193,  1.1187],
         [ 0.4337,  0.4337,  0.4337,  ...,  1.0502,  0.3823,  0.7591],
         ...,
         [-0.8164, -0.8849, -0.9020,  ...,  0.7419,  0.4337,  0.0741],
         [-0.8678, -0.9534, -0.9192,  ...,  0.8961,  0.6221,  0.3481],
         [-0.9705, -0.9877, -0.8678,  ...,  0.5707,  0.7933,  0.5878]],

        [[ 0.1001,  0.1001,  0.1001,  ...,  0.6429,  0.2052,  0.8179],
         [ 0.1001,  0.1001,  0.1001,  .

In [7]:
from easydict import EasyDict

def load_net(exp_dir,exp_name,checkpoint_path):
    cfg_path = f'../mmdetection/configs/{exp_dir}/{exp_name}'
    config = cfg_path
    
    args = EasyDict({
        'config' : config,
        'checkpoint': checkpoint_path,
        'score_thr' :0.001,
        'iou_thr': 0.5, 
        'work_dir': None,
        'seed': 42
    })

    cfg = Config.fromfile(args.config)
    seed = args.seed
    cfg.gpu_ids = [1]

    cfg.work_dir = osp.join('../work_dirs',osp.splitext(osp.basename(args.config))[0])
    print(cfg.work_dir)
    cfg.data.test.test_mode = True
    

    cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
    cfg.model.train_cfg = None
    cfg.model.test_cfg.rcnn.score_thr = args.score_thr
    cfg.model.test_cfg.rcnn.iou_threshold = args.iou_thr
    
    cfg.data.test['ann_file'] = os.path.join('../',cfg.data.test['ann_file'])
    print(cfg.data.test)
    
    dataset = build_dataset(cfg.data.test)
    data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

    epoch = args.checkpoint
    print(f'epoch: {epoch}')
    # checkpoint path
    checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

    model = build_detector(cfg.model, test_cfg=cfg.get(
        'test_cfg'))  # build detector
    checkpoint = load_checkpoint(
        model, checkpoint_path, map_location='cpu')  # ckpt load
    
    model = MMDataParallel(model.cuda(), device_ids=[0])
    
    num_classes = 10  
    model.eval()
    gc.collect()
    print('====================')
    print('model load finished\n')
    return model

models = [
    load_net(exp_dir='sparse_rcnn_r50_fpn_300_proposals_crop_mstrain_704-1024_36_coco',
        exp_name='sparse_rcnn_r50_fpn_300_proposals_crop_mstrain_704-1024_36_coco.py',
            checkpoint_path='best_bbox_mAP_50_epoch_22'),
    load_net(exp_dir='exp1',
            exp_name='cascadeRCNN_r50_fpn.py',
            checkpoint_path='best_bbox_mAP_50_epoch_23'),
]

../work_dirs/sparse_rcnn_r50_fpn_300_proposals_crop_mstrain_704-1024_36_coco
{'type': 'CocoDataset', 'ann_file': '../dataset2/test.json', 'img_prefix': 'dataset2/', 'pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (512, 512), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}], 'classes': ('General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing'), 'test_mode': True}
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
epoch: best_bbox_mAP_50_epoch_22
load checkpoint from local path: ../work_dirs/sparse_rcnn_r50_fpn_300_proposals_crop_mstrain_704-1024_36_coco/best_bbox_mAP_50_epoch_22.pth
model 

In [38]:
# !pip install ensemble-boxes

     |████████████████████████████████| 3.3 MB 46.6 MB/s 
     |████████████████████████████████| 34.5 MB 53.0 MB/s 


In [8]:
from ensemble_boxes import *

device = torch.device('cuda:0')

def make_ensemble_predictions(images):
    images = list(image.to(device) for image in images)    
    result = []
    for net in models:
        outputs = net(images)
        result.append(outputs)
    return result

def run_wbf(predictions, image_index, image_size=512, iou_thr=0.55, skip_box_thr=0.7, weights=None):
    boxes = [prediction[image_index]['boxes'].data.cpu().numpy()/(image_size-1) for prediction in predictions]
    scores = [prediction[image_index]['scores'].data.cpu().numpy() for prediction in predictions]
    labels = [np.ones(prediction[image_index]['scores'].shape[0]) for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

In [52]:
import matplotlib.pyplot as plt

for j, (images, image_ids) in enumerate(data_loader):
    if j > 0:
        break
predictions = make_ensemble_predictions(images)

i = 1
sample = images[i].permute(1,2,0).cpu().numpy()
boxes, scores, labels = run_wbf(predictions, image_index=i)
boxes = boxes.astype(np.int32).clip(min=0, max=511)

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 2)
    
ax.set_axis_off()
ax.imshow(sample);

ValueError: too many values to unpack (expected 2)

In [14]:
data = data_loader[0].dataset
data['img']

TypeError: list indices must be integers or slices, not str

In [ ]:
# WBF 적용되지 않은 결과

fig, axes = plt.subplots(2, 5)
fig.set_size_inches(15, 8)
color = [(0,0,255),(255,0,255)]



for i, (images, image_ids) in enumerate(test_loader):
    predictions = make_ensemble_predictions(images)

    img = cv2.imread(f"/kaggle/input/global-wheat-detection/test/{image_ids[0]}.jpg", cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    for folds in range(len(predictions)):
        for index in range(len(predictions[folds][0]['boxes'])):
            
            # bbox 값의 범위를 이미지 사이즈(1024)에 맞춰준다.
            bbox = (predictions[folds][0]['boxes'][index]*2)
            bbox = bbox.data.cpu().numpy().astype(np.int32).clip(min=0, max=1023)
            xmin, ymin, xmax, ymax = int(float(bbox[0])), int(float(bbox[1])), int(float(bbox[2])), int(float(bbox[3]))
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), color[folds], 3)
    axes[i//5, i%5].imshow(img)
    axes[i//5, i%5].set_title(image_ids[0])
    axes[i//5, i%5].axis('off')